In [1]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import pandas as pd
import functools

from scipy import sparse
from scipy.sparse import linalg
import time
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, mapping
from math import sqrt

from shapely.ops import cascaded_union
pd.set_option('precision', 5)
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
import image_city_functions as ic

In [3]:
# initialise path, names, etc.

city_name = 'London'
folder_ouptut ='Outputs/'+city_name+'/'
epsg = 27700
crs = {'init': 'epsg:27700', 'no_defs': True}

In [4]:
## Boston

# tmp = gpd.read_file('C:/Users/g_filo01/sciebo/Scripts/Tools/Outputs/bh_2002_26986.shp').to_crs(epsg=26986)
# buffer = gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/buffer_4000.shp').to_crs(epsg=26986)
# buildings_clip.to_file(driver='ESRI Shapefile',filename="Outputs/tmp/buildings_sight.shp")
# buildings = tmp[tmp.geometry.within(buffer.geometry.loc[0])]

In [5]:
## London
buildings = gpd.read_file("C:/Users/g_filo01/sciebo/GIS Data/London/Buildings/London_buildingsBase.shp")
obstructions = gpd.read_file("Outputs/tmp/"+city_name+"_obstructions.shp")

In [ ]:
all_buildings = gpd.read_file()
polygon_w = gpd.read_file()

buildings, obstructions = ic.select_buildings()
# all buildings in 4 km buffer to use as obstacles in ArcScene and for some analyis in this notebook
obstructions.to_file(driver='ESRI Shapefile', filename= "Outputs/tmp/"+city_name+"_obstructions.shp")

In [ ]:
# to simplify 
# when a building is divided in several polygons import in QGIS --> dissolve, single to multipart, join attributes by location
# buildings.to_file(driver='ESRI Shapefile',filename="Outputs/tmp/building_to_dissolve.shp")
# load shapefile again
# buildings = gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/bh_26986_simplified.shp').to_crs(epsg=26986)

## Structural Properties

In [6]:
%%time

streets = gpd.read_file("Outputs/"+city_name+"/"+city_name+"_paths.shp")
buildings = ic.structural_properties(buildings, obstructions, streets)

Wall time: 2min 41s


## Prominence

# Landuse

In [ ]:
# to be performed on the larger set of buildings
buildings['land_use'] = 'NaN'

## Parcels 

In [ ]:
# loading official data

directory_lu = 'C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/Parcels_2016/parcels.shp'
lu = (gpd.read_file(directory_lu).to_crs(epsg=epsg))

In [ ]:
parcels = parcels[['geometry', 'LU']]
list(parcels['LU'].unique())

In [ ]:
residential = ['Residential Three Family', 'Residential Two Family', 'Residential Single Family',
               'Apartments 4-6 Units', 'Apartments 7 Units above', 'Residential Land', 'Residential Condo Unit',
                 'Condominium Master', 'Mixed Use Res/Comm']
               
commercial = ['Commercial Land','Commercial Condo Unit','Commercial']
agricultural = ['Agricultural/Horticultural']
              
industrial = ['Industrial']
drop = ['Condominium Parking', 'Exempt 121A', 'XX', 'Exempt']

In [ ]:
parcels = parcels[~parcels['LU'].isin(drop)]

In [ ]:
parcels['LU'] = parcels['LU'].map( lambda x: 'residential' if x in residential
                                                          else 'commercial' if x in commercial
                                                          else 'residential' if x in residential
                                                          else 'industrial' if x in industrial
                                                          else 'agricultural'if x in agricultural 
                                                          else x)

In [ ]:
parcels['nr'] = 1

In [ ]:
%%time

sindex = parcels.sindex

for row in buildings.itertuples():
    
    g = row[2] #geometry
    possible_matches_index = list(sindex.intersection(g.bounds))
    possible_matches = parcels.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(g)]
        
    if (len(precise_matches)==0): continue
    
    else:
        
        land_use = precise_matches.groupby(['LU'],as_index=False)['nr'].sum().sort_values(by='nr',
                                    ascending=False).reset_index()
        main_use = land_use['LU'].loc[0]
        buildings.set_value(row[0], 'land_use', main_use)

## OpenStreetMap POI

In [ ]:
#land use from OSM POI

POI = gpd.read_file('Outputs/Boston/POI_boston.shp').to_crs(epsg=26986)
POI.head()

In [ ]:
list(POI['type'].unique())

In [ ]:
university = ['university', 'college', 'research']
commercial = ['bank', 'service',  'commercial',  'retail',  'pharmacy', 'commercial;educa', 'shop', 'Commercial',
                       'supermarket', 'offices', 'foundation', 'office']
residential = [ 'apartments']
attractions = ['museum',  'castle', 'cathedral', 'attraction','aquarium', 'monument',  'gatehouse',  'terrace']
manufacturing = []
hospitality = [ 'hotel',  'hostel', 'guest_house']
eating_drinking = [ 'restaurant', 'fast_food', 'cafe', 'bar',  'pub']
public = ['post_office', 'townhall', 'public_building',  'library','civic', 'courthouse','public',  'embassy']
library = ['library']
sport = ['stadium']
entertainment = [ 'exhibition_centr','theatre', 'cinema']
education = ['school', 'kindergarten']
religious = ['church', 'place_of_worship','convent', 'rectory']
emergency_service = [ 'fire_station','police']
transport = [ 'station', 'train_station']
medical_care = ['hospital', 'doctors', 'dentist','clinic','veterinary']
industrial = [ 'industrial', 'factory', 'construction']
cultural = [ 'club_house','gallery', 'arts_centre']
military = ['general aviation']

drop = ['subway_entrance', 'dormitory', 'roof', 'garage', 'storage_tank', 'shed', 'silo',  'parking',  'toilets','bus_station',
  'picnic_site','hut', 'information',  'viewpoint', 'atm', 'building', 'books',  'tower',  'terrace',  'canopy', 'smokestack',  
 'greenhouse', 'fuel']

In [ ]:
POI = POI[~POI['type'].isin(drop)]

In [ ]:
POI['type'] = POI['type'].map( lambda x: 'university' if x in university
                                                          else 'commercial' if x in commercial
                                                          else 'residential' if x in residential
                                                          else 'attractions' if x in attractions
                                                          else 'library' if x in library
                                                          else 'manufacturing' if x in manufacturing
                                                          else 'hospitality' if x in hospitality
                                                          else 'eating_drinking' if x in eating_drinking
                                                          else 'public' if x in public
                                                          else 'sport' if x in sport
                                                          else 'entertainment' if x in entertainment
                                                          else 'education' if x in education
                                                          else 'religious' if x in religious
                                                          else 'emergency_service' if x in emergency_service
                                                          else 'industrial' if x in industrial
                                                          else 'cultural' if x in cultural
                                                          else 'transport' if x in transport
                                                          else 'medical_care' if x in medical_care
                                                          else 'military' if x in military
                                                          else x)

In [ ]:
POI['nr'] = 1

In [ ]:
%%time

sindex = POI.sindex

for row in buildings.itertuples():
    g = row[2] #geometry
    
    possible_matches_index = list(sindex.intersection(g.bounds))
    possible_matches = POI.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(g)]
    
    
    if (len(precise_matches)==0): continue
    else:
        land_use = possible_matches.groupby(['type'],as_index=False)['nr'].sum().sort_values(by='nr',
                                    ascending=False).reset_index()
        main_use = land_use['type'].loc[0]
        buildings.set_value(row[0], 'land_use', main_use)

## Other data from Boston

In [ ]:
# libraries and schools

schools = (gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/other/schools.shp'
                        ).to_crs(epsg=26986))

universities = (gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/other/universities.shp'
                        ).to_crs(epsg=26986))

p_schools = (gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/other/private_schools.shp'
                        ).to_crs(epsg=26986))

libraries = (gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/other/libraries.shp'
                        ).to_crs(epsg=26986))

pools = (gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/other/pools.shp'
                        ).to_crs(epsg=26986))

police = (gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/other/police_stations.shp'
                        ).to_crs(epsg=26986))

In [ ]:
%%time

df_list = [schools, p_schools, libraries, universities, pools, police]
classification = ['education','education', 'library', 'university','sport', 'emergence_service']

for i, t in enumerate(df_list):
    sindex = t.sindex

    for row in buildings.itertuples():
        g = row[2] #geometry
    
        possible_matches_index = list(sindex.intersection(g.bounds))
        possible_matches = t.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(g)]
        
        if (len(precise_matches)==0): continue
        else:

            buildings.set_value(row[0], 'land_use', classification[i])

In [ ]:
buildings.land_use.fillna('residential', inplace=True)

In [ ]:
# buildings['land_use'].astype(object).fillna('residential', inplace=True)
buildings['land_use'].replace(to_replace='NaN', value='residential', inplace=True)
buildings.tail(10)

In [ ]:
buildings['land_use'].isnull().values.any()

In [ ]:
%%time

buildings['nr']=1
sindex = buildings.sindex
buildings['pragmatic_meaning']= 0.0

for row in buildings.itertuples():
    g = row[2] #geometry
    b = g.buffer(200)
    use = row[5]
    
    possible_matches_index = list(sindex.intersection(b.bounds))
    possible_matches = buildings.iloc[possible_matches_index]
#     precise_matches = buildings[buildings.intersects(g)]
        
    neigh = possible_matches.groupby(['land_use'], as_index=True)['nr'].sum()
    Nj = neigh.loc[use]
    #Pj = Nj/N
    
    Pj = 1-(Nj/possible_matches['nr'].sum())
    buildings.set_value(row[0], 'pragmatic_meaning', Pj)

In [ ]:
buildings.tail()

In [ ]:
buildings_clip = pd.merge(buildings_clip, buildings[['BUILD_ID','pragmatic_meaning']], on='BUILD_ID', how='left')

In [ ]:
buildings_clip.head()

# Cultural importance

In [ ]:
#load historical polygons

historic = (gpd.read_file('C:/Users/g_filo01/sciebo/GIS Data/Boston/Buildings/Data_sources/other/historic_landmarks.shp')).to_crs(epsg=26986)

In [ ]:
# historic = historic.dropna(subset=['geometry'])
# historic = historic.to_crs(epsg=26986)

In [ ]:
%%time

sindex = historic.sindex 
buildings_clip['cultural_meaning'] = 0

for row in buildings_clip.itertuples():
    g = row[2] #geometry
    possible_matches_index = list(sindex.intersection(g.bounds))
    possible_matches = historic.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(g)]
        
    if (len(precise_matches)==0): continue
    else: cm = precise_matches.groupby([score], as_index=True)['nr'].sum()

        buildings_clip.set_value(row[0], 'cultural_meaning', cm)

# Visibility 

In [ ]:
col = ['area', 'extension', 'facade_area', 'height', 'pragmatic_meaning', 'prominence','cultural_meaning', 'visibility']

for i in col:
       tmp[i+"_scaled"] = (tmp[i]-tmp[i].min())/(tmp[i].max()-tmp[i].min())

In [ ]:
col = ['neighbours', 'distance_road']

for i in col:
       tmp[i+"_scaled"] = 1-(tmp[i]-tmp[i].min())/(tmp[i].max()-tmp[i].min())

In [ ]:
tmp_scaled = tmp[['BUILD_ID','geometry','area_scaled','extension_scaled','facade_area_scaled',
                 'height_scaled','prominence_scaled','cultural_meaning_scaled',
                 'neighbours_scaled','distance_road_scaled','pragmatic_meaning_scaled', 'visibility_scaled']].copy()

In [ ]:
tmp_scaled['visual']=(tmp_scaled['facade_area_scaled']*30+tmp_scaled['height_scaled']*20
                      +tmp_scaled['visibility_scaled']*50)/100

In [ ]:
tmp_scaled['structural']=(tmp_scaled['extension_scaled']*30+tmp_scaled['neighbours_scaled']*20+
                   tmp_scaled['prominence_scaled']*30+tmp_scaled['distance_road_scaled']*20)/100

In [ ]:
col = ['visual', 'structural']

for i in col:
       tmp_scaled[i+"_scaled"] = (tmp_scaled[i]-tmp_scaled[i].min())/(tmp_scaled[i].max()-tmp_scaled[i].min())

In [ ]:
weights = tmp_scaled[['BUILD_ID','geometry','visual_scaled','structural_scaled',
                      'cultural_meaning_scaled', 'pragmatic_meaning_scaled']].copy()

In [ ]:
weights.crs

In [ ]:
weights['score']=(weights['visual_scaled']*50+weights['structural_scaled']*30+weights['cultural_meaning_scaled']*10+
                  weights['pragmatic_meaning_scaled']*10)/100

In [ ]:
weights['score_scaled']=(weights['score']-weights['score'].min())/(weights['score'].max()-weights['score'].min())

In [ ]:
weights.rename(columns={'visual_scaled': 'visual',
                        'structural_scaled':'structural',
                        'cultural_meaning_scaled': 'cultural',
                        'pragmatic_meaning_scaled':'pragmatic'}, inplace=True)

In [ ]:
weights.head(10)

In [ ]:
weights.to_file(driver='ESRI Shapefile',filename="Outputs/Boston/boston_landmark_scores.shp")